In [3]:
# x86_desktop 1.5GHz -> 3.0GHz
import pandas as pd
import math
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, VotingRegressor
from sklearn.metrics import r2_score
import pandas as pd
import math
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, VotingRegressor
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_percentage_error

def process_csv_feautures(file_paths, chunk_size):
    all_features = []
    for file_path in file_paths:
        df = pd.read_csv(file_path + '_' + str(chunk_size))
        df = df.apply(pd.to_numeric, errors='coerce').fillna(0)
        features = []
        for index, row in df.iterrows():
            inst = row['instructions:u']
            cyc = row['cpu-cycles:u']
            ipc = inst/cyc
            features.append([
                row['instructions:u'],
                row['cpu-cycles:u'],
                row['br_inst_retired.all_branches_pebs:u'],
                row['br_misp_retired.all_branches_pebs:u'],
                row['L1-dcache-load-misses:u'],
                row['L1-dcache-loads:u'],
                row['L1-dcache-stores:u'],
                row['L1-icache-load-misses:u'],
                row['LLC-load-misses:u'],
                row['LLC-loads:u'],
                row['LLC-store-misses:u'],
                row['LLC-stores:u'],
                row['fp_arith_inst_retired.scalar_single:u'],
                row['dTLB-load-misses:u'],
                row['dTLB-loads:u'],
                row['dTLB-store-misses:u'],
                row['dTLB-stores:u'],
                row['iTLB-load-misses:u'],
                row['iTLB-loads:u'],
                row['mem_load_retired.l1_hit:u'],
                row['mem_load_retired.l1_miss:u'],
                row['mem_load_retired.l2_hit:u'],
                row['mem_load_retired.l2_miss:u'],
                row['mem_load_retired.l3_hit:u'],
                row['mem_load_retired.l3_miss:u'],
                ipc])
        all_features.extend(features)
    return all_features


def process_csv_ipc(file_paths, chunk_size):
    all_ipc_values = []
    for file_path in file_paths:
        df = pd.read_csv(file_path + '_' + str(chunk_size))
        df = df.apply(pd.to_numeric, errors='coerce').fillna(0)
        for _, row in df.iterrows():
            all_ipc_values.append(row['instructions:u'] / row['cpu-cycles:u'])  
    return all_ipc_values

one_ghz_files = [
    '../../Data/traces/inst_aligned_traces/x86_desktop_homogeneous/x86_desktop_homogeneous_1.5GHz_500',
    '../../Data/traces/inst_aligned_traces/x86_desktop_homogeneous/x86_desktop_homogeneous_1.5GHz_502',
    '../../Data/traces/inst_aligned_traces/x86_desktop_homogeneous/x86_desktop_homogeneous_1.5GHz_505',
    '../../Data/traces/inst_aligned_traces/x86_desktop_homogeneous/x86_desktop_homogeneous_1.5GHz_520',
    '../../Data/traces/inst_aligned_traces/x86_desktop_homogeneous/x86_desktop_homogeneous_1.5GHz_523',
    '../../Data/traces/inst_aligned_traces/x86_desktop_homogeneous/x86_desktop_homogeneous_1.5GHz_525',
    '../../Data/traces/inst_aligned_traces/x86_desktop_homogeneous/x86_desktop_homogeneous_1.5GHz_531',
    '../../Data/traces/inst_aligned_traces/x86_desktop_homogeneous/x86_desktop_homogeneous_1.5GHz_541',
    '../../Data/traces/inst_aligned_traces/x86_desktop_homogeneous/x86_desktop_homogeneous_1.5GHz_548',
    '../../Data/traces/inst_aligned_traces/x86_desktop_homogeneous/x86_desktop_homogeneous_1.5GHz_557',
    '../../Data/traces/inst_aligned_traces/x86_desktop_homogeneous/x86_desktop_homogeneous_1.5GHz_503',
    '../../Data/traces/inst_aligned_traces/x86_desktop_homogeneous/x86_desktop_homogeneous_1.5GHz_507',
    '../../Data/traces/inst_aligned_traces/x86_desktop_homogeneous/x86_desktop_homogeneous_1.5GHz_508',
    '../../Data/traces/inst_aligned_traces/x86_desktop_homogeneous/x86_desktop_homogeneous_1.5GHz_510',
    '../../Data/traces/inst_aligned_traces/x86_desktop_homogeneous/x86_desktop_homogeneous_1.5GHz_511',
    '../../Data/traces/inst_aligned_traces/x86_desktop_homogeneous/x86_desktop_homogeneous_1.5GHz_519',
    '../../Data/traces/inst_aligned_traces/x86_desktop_homogeneous/x86_desktop_homogeneous_1.5GHz_521',
    '../../Data/traces/inst_aligned_traces/x86_desktop_homogeneous/x86_desktop_homogeneous_1.5GHz_527',
    '../../Data/traces/inst_aligned_traces/x86_desktop_homogeneous/x86_desktop_homogeneous_1.5GHz_538',
    '../../Data/traces/inst_aligned_traces/x86_desktop_homogeneous/x86_desktop_homogeneous_1.5GHz_544',
    '../../Data/traces/inst_aligned_traces/x86_desktop_homogeneous/x86_desktop_homogeneous_1.5GHz_549',
    '../../Data/traces/inst_aligned_traces/x86_desktop_homogeneous/x86_desktop_homogeneous_1.5GHz_554',
]
    
three_ghz_files = [
    '../../Data/traces/inst_aligned_traces/x86_desktop_homogeneous/x86_desktop_homogeneous_3.0GHz_500',
    '../../Data/traces/inst_aligned_traces/x86_desktop_homogeneous/x86_desktop_homogeneous_3.0GHz_502',
    '../../Data/traces/inst_aligned_traces/x86_desktop_homogeneous/x86_desktop_homogeneous_3.0GHz_505',
    '../../Data/traces/inst_aligned_traces/x86_desktop_homogeneous/x86_desktop_homogeneous_3.0GHz_520',
    '../../Data/traces/inst_aligned_traces/x86_desktop_homogeneous/x86_desktop_homogeneous_3.0GHz_523',
    '../../Data/traces/inst_aligned_traces/x86_desktop_homogeneous/x86_desktop_homogeneous_3.0GHz_525',
    '../../Data/traces/inst_aligned_traces/x86_desktop_homogeneous/x86_desktop_homogeneous_3.0GHz_531',
    '../../Data/traces/inst_aligned_traces/x86_desktop_homogeneous/x86_desktop_homogeneous_3.0GHz_541',
    '../../Data/traces/inst_aligned_traces/x86_desktop_homogeneous/x86_desktop_homogeneous_3.0GHz_548',
    '../../Data/traces/inst_aligned_traces/x86_desktop_homogeneous/x86_desktop_homogeneous_3.0GHz_557',
    '../../Data/traces/inst_aligned_traces/x86_desktop_homogeneous/x86_desktop_homogeneous_3.0GHz_503',
    '../../Data/traces/inst_aligned_traces/x86_desktop_homogeneous/x86_desktop_homogeneous_3.0GHz_507',
    '../../Data/traces/inst_aligned_traces/x86_desktop_homogeneous/x86_desktop_homogeneous_3.0GHz_508',
    '../../Data/traces/inst_aligned_traces/x86_desktop_homogeneous/x86_desktop_homogeneous_3.0GHz_510',
    '../../Data/traces/inst_aligned_traces/x86_desktop_homogeneous/x86_desktop_homogeneous_3.0GHz_511',
    '../../Data/traces/inst_aligned_traces/x86_desktop_homogeneous/x86_desktop_homogeneous_3.0GHz_519',
    '../../Data/traces/inst_aligned_traces/x86_desktop_homogeneous/x86_desktop_homogeneous_3.0GHz_521',
    '../../Data/traces/inst_aligned_traces/x86_desktop_homogeneous/x86_desktop_homogeneous_3.0GHz_527',
    '../../Data/traces/inst_aligned_traces/x86_desktop_homogeneous/x86_desktop_homogeneous_3.0GHz_538',
    '../../Data/traces/inst_aligned_traces/x86_desktop_homogeneous/x86_desktop_homogeneous_3.0GHz_544',
    '../../Data/traces/inst_aligned_traces/x86_desktop_homogeneous/x86_desktop_homogeneous_3.0GHz_549',
    '../../Data/traces/inst_aligned_traces/x86_desktop_homogeneous/x86_desktop_homogeneous_3.0GHz_554',
]

chunks = [200, 400, 600, 800, 1000]
for chunk_size in chunks:
    in_order_hardware_counters_features = process_csv_feautures(one_ghz_files, chunk_size)
    out_of_order_ipc = process_csv_ipc(three_ghz_files, chunk_size)
    print('chunk_size: ' + str(chunk_size))
    
    data = {
        'In Order': in_order_hardware_counters_features,
        'Out of Order': out_of_order_ipc
    }
    
    length = min(len(in_order_hardware_counters_features), len(out_of_order_ipc))
    
    # Shorten the longen array
    in_order_hardware_counters_features = in_order_hardware_counters_features[:length]
    out_of_order_ipc = out_of_order_ipc[:length]
    
    # separate features and target
    X = pd.DataFrame(
        in_order_hardware_counters_features, 
        columns=[
            'instructions:u',
            'cpu-cycles:u',
            'br_inst_retired.all_branches_pebs:u',
            'br_misp_retired.all_branches_pebs:u',
            'L1-dcache-load-misses:u',
            'L1-dcache-loads:u',
            'L1-dcache-stores:u',
            'L1-icache-load-misses:u',
            'LLC-load-misses:u',
            'LLC-loads:u',
            'LLC-store-misses:u',
            'LLC-stores:u',
            'fp_arith_inst_retired.scalar_single:u',
            'dTLB-load-misses:u',
            'dTLB-loads:u',
            'dTLB-store-misses:u',
            'dTLB-stores:u',
            'iTLB-load-misses:u',
            'iTLB-loads:u',
            'mem_load_retired.l1_hit:u',
            'mem_load_retired.l1_miss:u',
            'mem_load_retired.l2_hit:u',
            'mem_load_retired.l2_miss:u',
            'mem_load_retired.l3_hit:u',
            'mem_load_retired.l3_miss:u',
            'ipc'])  # Features
    y = pd.Series(out_of_order_ipc) 
    
    # split 
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
    
    rf_model = RandomForestRegressor(n_estimators=100)
    gb_model = GradientBoostingRegressor(n_estimators=100)
    ensemble_model = gb_model
    #ensemble_model = VotingRegressor([('rf', rf_model), ('gb', gb_model)])
    ensemble_model.fit(X_train, y_train)
    
    ensemble_predictions = ensemble_model.predict(X_test)
    # Calculate R²
    ensemble_r2 = r2_score(y_test, ensemble_predictions)
    
    #bruh wut
    train_predictions = ensemble_model.predict(X_train)
    test_predictions = ensemble_model.predict(X_test)
    
    # Calculate R² for training and testing sets
    train_r2 = r2_score(y_train, train_predictions)
    train_mape = mean_absolute_percentage_error(y_train, train_predictions)
    test_r2 = r2_score(y_test, test_predictions)
    test_mape = mean_absolute_percentage_error(y_test, test_predictions)
    
    
    print(f"Training R²: {train_r2}")
    print(f"Testing R²: {test_r2}")
    print(f"Training Mape: {train_mape}")
    print(f"Testing Mape: {test_mape}")


chunk_size: 200
Training R²: 0.9963263531777354
Testing R²: 0.9929805907509014
Training Mape: 0.014233832097817579
Testing Mape: 0.01705987253413798
chunk_size: 400
Training R²: 0.9099439213040819
Testing R²: 0.8761025153751767
Training Mape: 0.06240181898368015
Testing Mape: 0.0689495898198105
chunk_size: 600
Training R²: 0.8778215547428134
Testing R²: 0.8532235397643191
Training Mape: 0.07829842228604048
Testing Mape: 0.08358476838628244
chunk_size: 800
Training R²: 0.8539444891624861
Testing R²: 0.8099903661475885
Training Mape: 0.08922438591469664
Testing Mape: 0.09400138668511222
chunk_size: 1000
Training R²: 0.827839691163015
Testing R²: 0.826594445799173
Training Mape: 0.09974457523508087
Testing Mape: 0.10076695127847103


In [4]:
# x86_desktop 3.0GHz -> 1.5GHz
import pandas as pd
import math
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, VotingRegressor
from sklearn.metrics import r2_score
import pandas as pd
import math
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, VotingRegressor
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_percentage_error

def process_csv_feautures(file_paths, chunk_size):
    all_features = []
    for file_path in file_paths:
        df = pd.read_csv(file_path + '_' + str(chunk_size))
        df = df.apply(pd.to_numeric, errors='coerce').fillna(0)
        features = []
        for index, row in df.iterrows():
            inst = row['instructions:u']
            cyc = row['cpu-cycles:u']
            ipc = inst/cyc
            features.append([
                row['instructions:u'],
                row['cpu-cycles:u'],
                row['br_inst_retired.all_branches_pebs:u'],
                row['br_misp_retired.all_branches_pebs:u'],
                row['L1-dcache-load-misses:u'],
                row['L1-dcache-loads:u'],
                row['L1-dcache-stores:u'],
                row['L1-icache-load-misses:u'],
                row['LLC-load-misses:u'],
                row['LLC-loads:u'],
                row['LLC-store-misses:u'],
                row['LLC-stores:u'],
                row['fp_arith_inst_retired.scalar_single:u'],
                row['dTLB-load-misses:u'],
                row['dTLB-loads:u'],
                row['dTLB-store-misses:u'],
                row['dTLB-stores:u'],
                row['iTLB-load-misses:u'],
                row['iTLB-loads:u'],
                row['mem_load_retired.l1_hit:u'],
                row['mem_load_retired.l1_miss:u'],
                row['mem_load_retired.l2_hit:u'],
                row['mem_load_retired.l2_miss:u'],
                row['mem_load_retired.l3_hit:u'],
                row['mem_load_retired.l3_miss:u'],
                ipc])
        all_features.extend(features)
    return all_features


def process_csv_ipc(file_paths, chunk_size):
    all_ipc_values = []
    for file_path in file_paths:
        df = pd.read_csv(file_path + '_' + str(chunk_size))
        df = df.apply(pd.to_numeric, errors='coerce').fillna(0)
        for _, row in df.iterrows():
            all_ipc_values.append(row['instructions:u'] / row['cpu-cycles:u'])  
    return all_ipc_values

one_ghz_files = [
    '../../Data/traces/inst_aligned_traces/x86_desktop_homogeneous/x86_desktop_homogeneous_1.5GHz_500',
    '../../Data/traces/inst_aligned_traces/x86_desktop_homogeneous/x86_desktop_homogeneous_1.5GHz_502',
    '../../Data/traces/inst_aligned_traces/x86_desktop_homogeneous/x86_desktop_homogeneous_1.5GHz_505',
    '../../Data/traces/inst_aligned_traces/x86_desktop_homogeneous/x86_desktop_homogeneous_1.5GHz_520',
    '../../Data/traces/inst_aligned_traces/x86_desktop_homogeneous/x86_desktop_homogeneous_1.5GHz_523',
    '../../Data/traces/inst_aligned_traces/x86_desktop_homogeneous/x86_desktop_homogeneous_1.5GHz_525',
    '../../Data/traces/inst_aligned_traces/x86_desktop_homogeneous/x86_desktop_homogeneous_1.5GHz_531',
    '../../Data/traces/inst_aligned_traces/x86_desktop_homogeneous/x86_desktop_homogeneous_1.5GHz_541',
    '../../Data/traces/inst_aligned_traces/x86_desktop_homogeneous/x86_desktop_homogeneous_1.5GHz_548',
    '../../Data/traces/inst_aligned_traces/x86_desktop_homogeneous/x86_desktop_homogeneous_1.5GHz_557',
    '../../Data/traces/inst_aligned_traces/x86_desktop_homogeneous/x86_desktop_homogeneous_1.5GHz_503',
    '../../Data/traces/inst_aligned_traces/x86_desktop_homogeneous/x86_desktop_homogeneous_1.5GHz_507',
    '../../Data/traces/inst_aligned_traces/x86_desktop_homogeneous/x86_desktop_homogeneous_1.5GHz_508',
    '../../Data/traces/inst_aligned_traces/x86_desktop_homogeneous/x86_desktop_homogeneous_1.5GHz_510',
    '../../Data/traces/inst_aligned_traces/x86_desktop_homogeneous/x86_desktop_homogeneous_1.5GHz_511',
    '../../Data/traces/inst_aligned_traces/x86_desktop_homogeneous/x86_desktop_homogeneous_1.5GHz_519',
    '../../Data/traces/inst_aligned_traces/x86_desktop_homogeneous/x86_desktop_homogeneous_1.5GHz_521',
    '../../Data/traces/inst_aligned_traces/x86_desktop_homogeneous/x86_desktop_homogeneous_1.5GHz_527',
    '../../Data/traces/inst_aligned_traces/x86_desktop_homogeneous/x86_desktop_homogeneous_1.5GHz_538',
    '../../Data/traces/inst_aligned_traces/x86_desktop_homogeneous/x86_desktop_homogeneous_1.5GHz_544',
    '../../Data/traces/inst_aligned_traces/x86_desktop_homogeneous/x86_desktop_homogeneous_1.5GHz_549',
    '../../Data/traces/inst_aligned_traces/x86_desktop_homogeneous/x86_desktop_homogeneous_1.5GHz_554',
]
    
three_ghz_files = [
    '../../Data/traces/inst_aligned_traces/x86_desktop_homogeneous/x86_desktop_homogeneous_3.0GHz_500',
    '../../Data/traces/inst_aligned_traces/x86_desktop_homogeneous/x86_desktop_homogeneous_3.0GHz_502',
    '../../Data/traces/inst_aligned_traces/x86_desktop_homogeneous/x86_desktop_homogeneous_3.0GHz_505',
    '../../Data/traces/inst_aligned_traces/x86_desktop_homogeneous/x86_desktop_homogeneous_3.0GHz_520',
    '../../Data/traces/inst_aligned_traces/x86_desktop_homogeneous/x86_desktop_homogeneous_3.0GHz_523',
    '../../Data/traces/inst_aligned_traces/x86_desktop_homogeneous/x86_desktop_homogeneous_3.0GHz_525',
    '../../Data/traces/inst_aligned_traces/x86_desktop_homogeneous/x86_desktop_homogeneous_3.0GHz_531',
    '../../Data/traces/inst_aligned_traces/x86_desktop_homogeneous/x86_desktop_homogeneous_3.0GHz_541',
    '../../Data/traces/inst_aligned_traces/x86_desktop_homogeneous/x86_desktop_homogeneous_3.0GHz_548',
    '../../Data/traces/inst_aligned_traces/x86_desktop_homogeneous/x86_desktop_homogeneous_3.0GHz_557',
    '../../Data/traces/inst_aligned_traces/x86_desktop_homogeneous/x86_desktop_homogeneous_3.0GHz_503',
    '../../Data/traces/inst_aligned_traces/x86_desktop_homogeneous/x86_desktop_homogeneous_3.0GHz_507',
    '../../Data/traces/inst_aligned_traces/x86_desktop_homogeneous/x86_desktop_homogeneous_3.0GHz_508',
    '../../Data/traces/inst_aligned_traces/x86_desktop_homogeneous/x86_desktop_homogeneous_3.0GHz_510',
    '../../Data/traces/inst_aligned_traces/x86_desktop_homogeneous/x86_desktop_homogeneous_3.0GHz_511',
    '../../Data/traces/inst_aligned_traces/x86_desktop_homogeneous/x86_desktop_homogeneous_3.0GHz_519',
    '../../Data/traces/inst_aligned_traces/x86_desktop_homogeneous/x86_desktop_homogeneous_3.0GHz_521',
    '../../Data/traces/inst_aligned_traces/x86_desktop_homogeneous/x86_desktop_homogeneous_3.0GHz_527',
    '../../Data/traces/inst_aligned_traces/x86_desktop_homogeneous/x86_desktop_homogeneous_3.0GHz_538',
    '../../Data/traces/inst_aligned_traces/x86_desktop_homogeneous/x86_desktop_homogeneous_3.0GHz_544',
    '../../Data/traces/inst_aligned_traces/x86_desktop_homogeneous/x86_desktop_homogeneous_3.0GHz_549',
    '../../Data/traces/inst_aligned_traces/x86_desktop_homogeneous/x86_desktop_homogeneous_3.0GHz_554',
]

chunks = [200, 400, 600, 800, 1000]
for chunk_size in chunks:
    in_order_hardware_counters_features = process_csv_feautures(three_ghz_files, chunk_size)
    out_of_order_ipc = process_csv_ipc(one_ghz_files, chunk_size)
    print('chunk_size: ' + str(chunk_size))
    
    data = {
        'In Order': in_order_hardware_counters_features,
        'Out of Order': out_of_order_ipc
    }
    
    length = min(len(in_order_hardware_counters_features), len(out_of_order_ipc))
    
    # Shorten the longen array
    in_order_hardware_counters_features = in_order_hardware_counters_features[:length]
    out_of_order_ipc = out_of_order_ipc[:length]
    
    # separate features and target
    X = pd.DataFrame(
        in_order_hardware_counters_features, 
        columns=[
            'instructions:u',
            'cpu-cycles:u',
            'br_inst_retired.all_branches_pebs:u',
            'br_misp_retired.all_branches_pebs:u',
            'L1-dcache-load-misses:u',
            'L1-dcache-loads:u',
            'L1-dcache-stores:u',
            'L1-icache-load-misses:u',
            'LLC-load-misses:u',
            'LLC-loads:u',
            'LLC-store-misses:u',
            'LLC-stores:u',
            'fp_arith_inst_retired.scalar_single:u',
            'dTLB-load-misses:u',
            'dTLB-loads:u',
            'dTLB-store-misses:u',
            'dTLB-stores:u',
            'iTLB-load-misses:u',
            'iTLB-loads:u',
            'mem_load_retired.l1_hit:u',
            'mem_load_retired.l1_miss:u',
            'mem_load_retired.l2_hit:u',
            'mem_load_retired.l2_miss:u',
            'mem_load_retired.l3_hit:u',
            'mem_load_retired.l3_miss:u',
            'ipc'])  # Features
    y = pd.Series(out_of_order_ipc) 
    
    # split 
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
    
    rf_model = RandomForestRegressor(n_estimators=100)
    gb_model = GradientBoostingRegressor(n_estimators=100)
    ensemble_model = gb_model
    #ensemble_model = VotingRegressor([('rf', rf_model), ('gb', gb_model)])
    ensemble_model.fit(X_train, y_train)
    
    ensemble_predictions = ensemble_model.predict(X_test)
    # Calculate R²
    ensemble_r2 = r2_score(y_test, ensemble_predictions)
    
    #bruh wut
    train_predictions = ensemble_model.predict(X_train)
    test_predictions = ensemble_model.predict(X_test)
    
    # Calculate R² for training and testing sets
    train_r2 = r2_score(y_train, train_predictions)
    train_mape = mean_absolute_percentage_error(y_train, train_predictions)
    test_r2 = r2_score(y_test, test_predictions)
    test_mape = mean_absolute_percentage_error(y_test, test_predictions)
    
    
    print(f"Training R²: {train_r2}")
    print(f"Testing R²: {test_r2}")
    print(f"Training Mape: {train_mape}")
    print(f"Testing Mape: {test_mape}")


chunk_size: 200
Training R²: 0.996202323193595
Testing R²: 0.9887944178705724
Training Mape: 0.013656674822877867
Testing Mape: 0.01764075664587753
chunk_size: 400
Training R²: 0.9136147461866034
Testing R²: 0.853901091822853
Training Mape: 0.059664979367266044
Testing Mape: 0.07021612806837903
chunk_size: 600
Training R²: 0.8719004621522811
Testing R²: 0.8491642102258181
Training Mape: 0.07833170673036718
Testing Mape: 0.08346515383265929
chunk_size: 800
Training R²: 0.833490349765204
Testing R²: 0.8085559748026923
Training Mape: 0.093450099665782
Testing Mape: 0.09689944821037406
chunk_size: 1000
Training R²: 0.8087000645903302
Testing R²: 0.8030366685768764
Training Mape: 0.10250738523376719
Testing Mape: 0.10234795226506034


In [5]:
# x86_ecore 1.5GHz -> 3.0GHz
import pandas as pd
import math
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, VotingRegressor
from sklearn.metrics import r2_score
import pandas as pd
import math
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, VotingRegressor
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_percentage_error

def process_csv_feautures(file_paths, chunk_size):
    all_features = []
    for file_path in file_paths:
        df = pd.read_csv(file_path + '_' + str(chunk_size))
        df = df.apply(pd.to_numeric, errors='coerce').fillna(0)
        features = []
        for index, row in df.iterrows():
            inst = row['cpu_atom/instructions/u']
            cyc = row['cpu_atom/cpu-cycles/u']
            ipc = inst/cyc
            features.append([
            row['cpu_atom/instructions/u'],
            row['cpu_atom/cpu-cycles/u'],
            row['cpu_atom/br_inst_retired.all_branches/u'],
            row['cpu_atom/br_misp_retired.all_branches/u'],
            row['cpu_atom/cache-misses/u'],
            row['cpu_atom/L1-dcache-loads:u/'],
            row['cpu_atom/L1-dcache-stores:u/'],
            row['cpu_atom/L1-icache-load-misses:u/'],
            row['cpu_atom/LLC-load-misses:u/'],
            row['cpu_atom/LLC-loads:u/'],
            row['cpu_atom/LLC-store-misses:u/'],
            row['cpu_atom/LLC-stores:u/'],
            row['cpu_core/fp_arith_inst_retired.scalar_single:u/'],
            row['cpu_atom/dTLB-load-misses:u/'],
            row['cpu_atom/dTLB-loads:u/'],
            row['cpu_atom/dTLB-store-misses:u/'],
            row['cpu_atom/dTLB-stores:u/'],
            row['cpu_atom/iTLB-load-misses:u/'],
            row['cpu_atom/bus-cycles/u'],
            row['cpu_core/mem_load_retired.l1_hit:u/'],
            row['cpu_core/mem_load_retired.l1_miss:u/'],
            row['cpu_core/mem_load_retired.l2_hit:u/'],
            row['cpu_core/mem_load_retired.l2_miss:u/'],
            row['cpu_core/mem_load_retired.l3_hit:u/'],
            row['cpu_core/mem_load_retired.l3_miss:u/'],
                ipc])
        all_features.extend(features)
    return all_features


def process_csv_ipc(file_paths, chunk_size):
    all_ipc_values = []
    for file_path in file_paths:
        df = pd.read_csv(file_path + '_' + str(chunk_size))
        df = df.apply(pd.to_numeric, errors='coerce').fillna(0)
        for _, row in df.iterrows():
            all_ipc_values.append(row['cpu_atom/instructions/u'] / row['cpu_atom/cpu-cycles/u'])  
    return all_ipc_values

one_ghz_files = [
    '../../Data/traces/inst_aligned_traces/x86_desktop_heterogeneous/x86_desktop_heterogeneous_ecore_1.5GHz_500',
    '../../Data/traces/inst_aligned_traces/x86_desktop_heterogeneous/x86_desktop_heterogeneous_ecore_1.5GHz_502',
    '../../Data/traces/inst_aligned_traces/x86_desktop_heterogeneous/x86_desktop_heterogeneous_ecore_1.5GHz_505',
    '../../Data/traces/inst_aligned_traces/x86_desktop_heterogeneous/x86_desktop_heterogeneous_ecore_1.5GHz_520',
    '../../Data/traces/inst_aligned_traces/x86_desktop_heterogeneous/x86_desktop_heterogeneous_ecore_1.5GHz_523',
    '../../Data/traces/inst_aligned_traces/x86_desktop_heterogeneous/x86_desktop_heterogeneous_ecore_1.5GHz_525',
    '../../Data/traces/inst_aligned_traces/x86_desktop_heterogeneous/x86_desktop_heterogeneous_ecore_1.5GHz_531',
    '../../Data/traces/inst_aligned_traces/x86_desktop_heterogeneous/x86_desktop_heterogeneous_ecore_1.5GHz_541',
    '../../Data/traces/inst_aligned_traces/x86_desktop_heterogeneous/x86_desktop_heterogeneous_ecore_1.5GHz_548',
    '../../Data/traces/inst_aligned_traces/x86_desktop_heterogeneous/x86_desktop_heterogeneous_ecore_1.5GHz_557',
    '../../Data/traces/inst_aligned_traces/x86_desktop_heterogeneous/x86_desktop_heterogeneous_ecore_1.5GHz_503',
    '../../Data/traces/inst_aligned_traces/x86_desktop_heterogeneous/x86_desktop_heterogeneous_ecore_1.5GHz_507',
    '../../Data/traces/inst_aligned_traces/x86_desktop_heterogeneous/x86_desktop_heterogeneous_ecore_1.5GHz_508',
    '../../Data/traces/inst_aligned_traces/x86_desktop_heterogeneous/x86_desktop_heterogeneous_ecore_1.5GHz_510',
    '../../Data/traces/inst_aligned_traces/x86_desktop_heterogeneous/x86_desktop_heterogeneous_ecore_1.5GHz_511',
    '../../Data/traces/inst_aligned_traces/x86_desktop_heterogeneous/x86_desktop_heterogeneous_ecore_1.5GHz_519',
    '../../Data/traces/inst_aligned_traces/x86_desktop_heterogeneous/x86_desktop_heterogeneous_ecore_1.5GHz_521',
    '../../Data/traces/inst_aligned_traces/x86_desktop_heterogeneous/x86_desktop_heterogeneous_ecore_1.5GHz_527',
    '../../Data/traces/inst_aligned_traces/x86_desktop_heterogeneous/x86_desktop_heterogeneous_ecore_1.5GHz_538',
    '../../Data/traces/inst_aligned_traces/x86_desktop_heterogeneous/x86_desktop_heterogeneous_ecore_1.5GHz_544',
    '../../Data/traces/inst_aligned_traces/x86_desktop_heterogeneous/x86_desktop_heterogeneous_ecore_1.5GHz_549',
    '../../Data/traces/inst_aligned_traces/x86_desktop_heterogeneous/x86_desktop_heterogeneous_ecore_1.5GHz_554',
]
    
three_ghz_files = [
    '../../Data/traces/inst_aligned_traces/x86_desktop_heterogeneous/x86_desktop_heterogeneous_ecore_3.0GHz_500',
    '../../Data/traces/inst_aligned_traces/x86_desktop_heterogeneous/x86_desktop_heterogeneous_ecore_3.0GHz_502',
    '../../Data/traces/inst_aligned_traces/x86_desktop_heterogeneous/x86_desktop_heterogeneous_ecore_3.0GHz_505',
    '../../Data/traces/inst_aligned_traces/x86_desktop_heterogeneous/x86_desktop_heterogeneous_ecore_3.0GHz_520',
    '../../Data/traces/inst_aligned_traces/x86_desktop_heterogeneous/x86_desktop_heterogeneous_ecore_3.0GHz_523',
    '../../Data/traces/inst_aligned_traces/x86_desktop_heterogeneous/x86_desktop_heterogeneous_ecore_3.0GHz_525',
    '../../Data/traces/inst_aligned_traces/x86_desktop_heterogeneous/x86_desktop_heterogeneous_ecore_3.0GHz_531',
    '../../Data/traces/inst_aligned_traces/x86_desktop_heterogeneous/x86_desktop_heterogeneous_ecore_3.0GHz_541',
    '../../Data/traces/inst_aligned_traces/x86_desktop_heterogeneous/x86_desktop_heterogeneous_ecore_3.0GHz_548',
    '../../Data/traces/inst_aligned_traces/x86_desktop_heterogeneous/x86_desktop_heterogeneous_ecore_3.0GHz_557',
    '../../Data/traces/inst_aligned_traces/x86_desktop_heterogeneous/x86_desktop_heterogeneous_ecore_3.0GHz_503',
    '../../Data/traces/inst_aligned_traces/x86_desktop_heterogeneous/x86_desktop_heterogeneous_ecore_3.0GHz_507',
    '../../Data/traces/inst_aligned_traces/x86_desktop_heterogeneous/x86_desktop_heterogeneous_ecore_3.0GHz_508',
    '../../Data/traces/inst_aligned_traces/x86_desktop_heterogeneous/x86_desktop_heterogeneous_ecore_3.0GHz_510',
    '../../Data/traces/inst_aligned_traces/x86_desktop_heterogeneous/x86_desktop_heterogeneous_ecore_3.0GHz_511',
    '../../Data/traces/inst_aligned_traces/x86_desktop_heterogeneous/x86_desktop_heterogeneous_ecore_3.0GHz_519',
    '../../Data/traces/inst_aligned_traces/x86_desktop_heterogeneous/x86_desktop_heterogeneous_ecore_3.0GHz_521',
    '../../Data/traces/inst_aligned_traces/x86_desktop_heterogeneous/x86_desktop_heterogeneous_ecore_3.0GHz_527',
    '../../Data/traces/inst_aligned_traces/x86_desktop_heterogeneous/x86_desktop_heterogeneous_ecore_3.0GHz_538',
    '../../Data/traces/inst_aligned_traces/x86_desktop_heterogeneous/x86_desktop_heterogeneous_ecore_3.0GHz_544',
    '../../Data/traces/inst_aligned_traces/x86_desktop_heterogeneous/x86_desktop_heterogeneous_ecore_3.0GHz_549',
    '../../Data/traces/inst_aligned_traces/x86_desktop_heterogeneous/x86_desktop_heterogeneous_ecore_3.0GHz_554',
]

chunks = [200, 400, 600, 800, 1000]
for chunk_size in chunks:
    in_order_hardware_counters_features = process_csv_feautures(one_ghz_files, chunk_size)
    out_of_order_ipc = process_csv_ipc(three_ghz_files, chunk_size)
    print('chunk_size: ' + str(chunk_size))
    
    data = {
        'In Order': in_order_hardware_counters_features,
        'Out of Order': out_of_order_ipc
    }
    
    length = min(len(in_order_hardware_counters_features), len(out_of_order_ipc))
    
    # Shorten the longen array
    in_order_hardware_counters_features = in_order_hardware_counters_features[:length]
    out_of_order_ipc = out_of_order_ipc[:length]
    
    # separate features and target
    X = pd.DataFrame(
        in_order_hardware_counters_features, 
        columns=[
            'cpu_atom/instructions/u',
            'cpu_atom/cpu-cycles/u',
            'cpu_atom/br_inst_retired.all_branches/u',
            'cpu_atom/br_misp_retired.all_branches/u',
            'cpu_atom/cache-misses/u',
            'cpu_atom/L1-dcache-loads:u/',
            'cpu_atom/L1-dcache-stores:u/',
            'cpu_atom/L1-icache-load-misses:u/',
            'cpu_atom/LLC-load-misses:u/',
            'cpu_atom/LLC-loads:u/',
            'cpu_atom/LLC-store-misses:u/',
            'cpu_atom/LLC-stores:u/',
            'cpu_core/fp_arith_inst_retired.scalar_single:u/',
            'cpu_atom/dTLB-load-misses:u/',
            'cpu_atom/dTLB-loads:u/',
            'cpu_atom/dTLB-store-misses:u/',
            'cpu_atom/dTLB-stores:u/',
            'cpu_atom/iTLB-load-misses:u/',
            'cpu_atom/bus-cycles/u',
            'cpu_core/mem_load_retired.l1_hit:u/',
            'cpu_core/mem_load_retired.l1_miss:u/',
            'cpu_core/mem_load_retired.l2_hit:u/',
            'cpu_core/mem_load_retired.l2_miss:u/',
            'cpu_core/mem_load_retired.l3_hit:u/',
            'cpu_core/mem_load_retired.l3_miss:u/',
            'ipc'])  # Features
    y = pd.Series(out_of_order_ipc) 
    
    # split 
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
    
    rf_model = RandomForestRegressor(n_estimators=100)
    gb_model = GradientBoostingRegressor(n_estimators=100)
    ensemble_model = gb_model
    #ensemble_model = VotingRegressor([('rf', rf_model), ('gb', gb_model)])
    ensemble_model.fit(X_train, y_train)
    
    ensemble_predictions = ensemble_model.predict(X_test)
    # Calculate R²
    ensemble_r2 = r2_score(y_test, ensemble_predictions)
    
    #bruh wut
    train_predictions = ensemble_model.predict(X_train)
    test_predictions = ensemble_model.predict(X_test)
    
    # Calculate R² for training and testing sets
    train_r2 = r2_score(y_train, train_predictions)
    train_mape = mean_absolute_percentage_error(y_train, train_predictions)
    test_r2 = r2_score(y_test, test_predictions)
    test_mape = mean_absolute_percentage_error(y_test, test_predictions)
    
    
    print(f"Training R²: {train_r2}")
    print(f"Testing R²: {test_r2}")
    print(f"Training Mape: {train_mape}")
    print(f"Testing Mape: {test_mape}")


chunk_size: 200
Training R²: 0.9950393204019493
Testing R²: 0.9876096098085132
Training Mape: 0.018834438941343967
Testing Mape: 0.027896193919811754
chunk_size: 400
Training R²: 0.8795001929605407
Testing R²: 0.8387491002452775
Training Mape: 0.09428872808892086
Testing Mape: 0.104388272490954
chunk_size: 600
Training R²: 0.8343486882781036
Testing R²: 0.8125670896681473
Training Mape: 0.11712989968341803
Testing Mape: 0.12722657359187078
chunk_size: 800
Training R²: 0.7977440811149357
Testing R²: 0.7325888744205076
Training Mape: 0.13847026134608267
Testing Mape: 0.14820239237496402
chunk_size: 1000
Training R²: 0.7491878680303021
Testing R²: 0.7322763010333442
Training Mape: 0.16182357854830154
Testing Mape: 0.16403521428842077


In [7]:
# x86_ecore 3.0GHz -> 1.5GHz
import pandas as pd
import math
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, VotingRegressor
from sklearn.metrics import r2_score
import pandas as pd
import math
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, VotingRegressor
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_percentage_error

def process_csv_feautures(file_paths, chunk_size):
    all_features = []
    for file_path in file_paths:
        df = pd.read_csv(file_path + '_' + str(chunk_size))
        df = df.apply(pd.to_numeric, errors='coerce').fillna(0)
        features = []
        for index, row in df.iterrows():
            inst = row['cpu_atom/instructions/u']
            cyc = row['cpu_atom/cpu-cycles/u']
            ipc = inst/cyc
            features.append([
            row['cpu_atom/instructions/u'],
            row['cpu_atom/cpu-cycles/u'],
            row['cpu_atom/br_inst_retired.all_branches/u'],
            row['cpu_atom/br_misp_retired.all_branches/u'],
            row['cpu_atom/cache-misses/u'],
            row['cpu_atom/L1-dcache-loads:u/'],
            row['cpu_atom/L1-dcache-stores:u/'],
            row['cpu_atom/L1-icache-load-misses:u/'],
            row['cpu_atom/LLC-load-misses:u/'],
            row['cpu_atom/LLC-loads:u/'],
            row['cpu_atom/LLC-store-misses:u/'],
            row['cpu_atom/LLC-stores:u/'],
            row['cpu_core/fp_arith_inst_retired.scalar_single:u/'],
            row['cpu_atom/dTLB-load-misses:u/'],
            row['cpu_atom/dTLB-loads:u/'],
            row['cpu_atom/dTLB-store-misses:u/'],
            row['cpu_atom/dTLB-stores:u/'],
            row['cpu_atom/iTLB-load-misses:u/'],
            row['cpu_atom/bus-cycles/u'],
                ipc])
        all_features.extend(features)
    return all_features


def process_csv_ipc(file_paths, chunk_size):
    all_ipc_values = []
    for file_path in file_paths:
        df = pd.read_csv(file_path + '_' + str(chunk_size))
        df = df.apply(pd.to_numeric, errors='coerce').fillna(0)
        for _, row in df.iterrows():
            all_ipc_values.append(row['cpu_atom/instructions/u'] / row['cpu_atom/cpu-cycles/u'])  
    return all_ipc_values

one_ghz_files = [
    '../../Data/traces/inst_aligned_traces/x86_desktop_heterogeneous/x86_desktop_heterogeneous_ecore_1.5GHz_500',
    '../../Data/traces/inst_aligned_traces/x86_desktop_heterogeneous/x86_desktop_heterogeneous_ecore_1.5GHz_502',
    '../../Data/traces/inst_aligned_traces/x86_desktop_heterogeneous/x86_desktop_heterogeneous_ecore_1.5GHz_505',
    '../../Data/traces/inst_aligned_traces/x86_desktop_heterogeneous/x86_desktop_heterogeneous_ecore_1.5GHz_520',
    '../../Data/traces/inst_aligned_traces/x86_desktop_heterogeneous/x86_desktop_heterogeneous_ecore_1.5GHz_523',
    '../../Data/traces/inst_aligned_traces/x86_desktop_heterogeneous/x86_desktop_heterogeneous_ecore_1.5GHz_525',
    '../../Data/traces/inst_aligned_traces/x86_desktop_heterogeneous/x86_desktop_heterogeneous_ecore_1.5GHz_531',
    '../../Data/traces/inst_aligned_traces/x86_desktop_heterogeneous/x86_desktop_heterogeneous_ecore_1.5GHz_541',
    '../../Data/traces/inst_aligned_traces/x86_desktop_heterogeneous/x86_desktop_heterogeneous_ecore_1.5GHz_548',
    '../../Data/traces/inst_aligned_traces/x86_desktop_heterogeneous/x86_desktop_heterogeneous_ecore_1.5GHz_557',
    '../../Data/traces/inst_aligned_traces/x86_desktop_heterogeneous/x86_desktop_heterogeneous_ecore_1.5GHz_503',
    '../../Data/traces/inst_aligned_traces/x86_desktop_heterogeneous/x86_desktop_heterogeneous_ecore_1.5GHz_507',
    '../../Data/traces/inst_aligned_traces/x86_desktop_heterogeneous/x86_desktop_heterogeneous_ecore_1.5GHz_508',
    '../../Data/traces/inst_aligned_traces/x86_desktop_heterogeneous/x86_desktop_heterogeneous_ecore_1.5GHz_510',
    '../../Data/traces/inst_aligned_traces/x86_desktop_heterogeneous/x86_desktop_heterogeneous_ecore_1.5GHz_511',
    '../../Data/traces/inst_aligned_traces/x86_desktop_heterogeneous/x86_desktop_heterogeneous_ecore_1.5GHz_519',
    '../../Data/traces/inst_aligned_traces/x86_desktop_heterogeneous/x86_desktop_heterogeneous_ecore_1.5GHz_521',
    '../../Data/traces/inst_aligned_traces/x86_desktop_heterogeneous/x86_desktop_heterogeneous_ecore_1.5GHz_527',
    '../../Data/traces/inst_aligned_traces/x86_desktop_heterogeneous/x86_desktop_heterogeneous_ecore_1.5GHz_538',
    '../../Data/traces/inst_aligned_traces/x86_desktop_heterogeneous/x86_desktop_heterogeneous_ecore_1.5GHz_544',
    '../../Data/traces/inst_aligned_traces/x86_desktop_heterogeneous/x86_desktop_heterogeneous_ecore_1.5GHz_549',
    '../../Data/traces/inst_aligned_traces/x86_desktop_heterogeneous/x86_desktop_heterogeneous_ecore_1.5GHz_554',
]
    
three_ghz_files = [
    '../../Data/traces/inst_aligned_traces/x86_desktop_heterogeneous/x86_desktop_heterogeneous_ecore_3.0GHz_500',
    '../../Data/traces/inst_aligned_traces/x86_desktop_heterogeneous/x86_desktop_heterogeneous_ecore_3.0GHz_502',
    '../../Data/traces/inst_aligned_traces/x86_desktop_heterogeneous/x86_desktop_heterogeneous_ecore_3.0GHz_505',
    '../../Data/traces/inst_aligned_traces/x86_desktop_heterogeneous/x86_desktop_heterogeneous_ecore_3.0GHz_520',
    '../../Data/traces/inst_aligned_traces/x86_desktop_heterogeneous/x86_desktop_heterogeneous_ecore_3.0GHz_523',
    '../../Data/traces/inst_aligned_traces/x86_desktop_heterogeneous/x86_desktop_heterogeneous_ecore_3.0GHz_525',
    '../../Data/traces/inst_aligned_traces/x86_desktop_heterogeneous/x86_desktop_heterogeneous_ecore_3.0GHz_531',
    '../../Data/traces/inst_aligned_traces/x86_desktop_heterogeneous/x86_desktop_heterogeneous_ecore_3.0GHz_541',
    '../../Data/traces/inst_aligned_traces/x86_desktop_heterogeneous/x86_desktop_heterogeneous_ecore_3.0GHz_548',
    '../../Data/traces/inst_aligned_traces/x86_desktop_heterogeneous/x86_desktop_heterogeneous_ecore_3.0GHz_557',
    '../../Data/traces/inst_aligned_traces/x86_desktop_heterogeneous/x86_desktop_heterogeneous_ecore_3.0GHz_503',
    '../../Data/traces/inst_aligned_traces/x86_desktop_heterogeneous/x86_desktop_heterogeneous_ecore_3.0GHz_507',
    '../../Data/traces/inst_aligned_traces/x86_desktop_heterogeneous/x86_desktop_heterogeneous_ecore_3.0GHz_508',
    '../../Data/traces/inst_aligned_traces/x86_desktop_heterogeneous/x86_desktop_heterogeneous_ecore_3.0GHz_510',
    '../../Data/traces/inst_aligned_traces/x86_desktop_heterogeneous/x86_desktop_heterogeneous_ecore_3.0GHz_511',
    '../../Data/traces/inst_aligned_traces/x86_desktop_heterogeneous/x86_desktop_heterogeneous_ecore_3.0GHz_519',
    '../../Data/traces/inst_aligned_traces/x86_desktop_heterogeneous/x86_desktop_heterogeneous_ecore_3.0GHz_521',
    '../../Data/traces/inst_aligned_traces/x86_desktop_heterogeneous/x86_desktop_heterogeneous_ecore_3.0GHz_527',
    '../../Data/traces/inst_aligned_traces/x86_desktop_heterogeneous/x86_desktop_heterogeneous_ecore_3.0GHz_538',
    '../../Data/traces/inst_aligned_traces/x86_desktop_heterogeneous/x86_desktop_heterogeneous_ecore_3.0GHz_544',
    '../../Data/traces/inst_aligned_traces/x86_desktop_heterogeneous/x86_desktop_heterogeneous_ecore_3.0GHz_549',
    '../../Data/traces/inst_aligned_traces/x86_desktop_heterogeneous/x86_desktop_heterogeneous_ecore_3.0GHz_554',
]

chunks = [200, 400, 600, 800, 1000]
for chunk_size in chunks:
    in_order_hardware_counters_features = process_csv_feautures(three_ghz_files, chunk_size)
    out_of_order_ipc = process_csv_ipc(one_ghz_files, chunk_size)
    print('chunk_size: ' + str(chunk_size))
    
    data = {
        'In Order': in_order_hardware_counters_features,
        'Out of Order': out_of_order_ipc
    }
    
    length = min(len(in_order_hardware_counters_features), len(out_of_order_ipc))
    
    # Shorten the longen array
    in_order_hardware_counters_features = in_order_hardware_counters_features[:length]
    out_of_order_ipc = out_of_order_ipc[:length]
    
    # separate features and target
    X = pd.DataFrame(
        in_order_hardware_counters_features, 
        columns=[
            'cpu_atom/instructions/u',
            'cpu_atom/cpu-cycles/u',
            'cpu_atom/br_inst_retired.all_branches/u',
            'cpu_atom/br_misp_retired.all_branches/u',
            'cpu_atom/cache-misses/u',
            'cpu_atom/L1-dcache-loads:u/',
            'cpu_atom/L1-dcache-stores:u/',
            'cpu_atom/L1-icache-load-misses:u/',
            'cpu_atom/LLC-load-misses:u/',
            'cpu_atom/LLC-loads:u/',
            'cpu_atom/LLC-store-misses:u/',
            'cpu_atom/LLC-stores:u/',
            'cpu_core/fp_arith_inst_retired.scalar_single:u/',
            'cpu_atom/dTLB-load-misses:u/',
            'cpu_atom/dTLB-loads:u/',
            'cpu_atom/dTLB-store-misses:u/',
            'cpu_atom/dTLB-stores:u/',
            'cpu_atom/iTLB-load-misses:u/',
            'cpu_atom/bus-cycles/u',
            'ipc'])  # Features
    y = pd.Series(out_of_order_ipc) 
    
    # split 
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
    
    rf_model = RandomForestRegressor(n_estimators=100)
    gb_model = GradientBoostingRegressor(n_estimators=100)
    ensemble_model = gb_model
    #ensemble_model = VotingRegressor([('rf', rf_model), ('gb', gb_model)])
    ensemble_model.fit(X_train, y_train)
    
    ensemble_predictions = ensemble_model.predict(X_test)
    # Calculate R²
    ensemble_r2 = r2_score(y_test, ensemble_predictions)
    
    #bruh wut
    train_predictions = ensemble_model.predict(X_train)
    test_predictions = ensemble_model.predict(X_test)
    
    # Calculate R² for training and testing sets
    train_r2 = r2_score(y_train, train_predictions)
    train_mape = mean_absolute_percentage_error(y_train, train_predictions)
    test_r2 = r2_score(y_test, test_predictions)
    test_mape = mean_absolute_percentage_error(y_test, test_predictions)
    
    
    print(f"Training R²: {train_r2}")
    print(f"Testing R²: {test_r2}")
    print(f"Training Mape: {train_mape}")
    print(f"Testing Mape: {test_mape}")


chunk_size: 200
Training R²: 0.9944798171590706
Testing R²: 0.9879487176927279
Training Mape: 0.01915509345322428
Testing Mape: 0.02520709294463617
chunk_size: 400
Training R²: 0.8771443597310266
Testing R²: 0.8342044567789726
Training Mape: 0.08950035233292211
Testing Mape: 0.09930994876766246
chunk_size: 600
Training R²: 0.8268238397322668
Testing R²: 0.7893245504635086
Training Mape: 0.11525080543757335
Testing Mape: 0.12299677437271077
chunk_size: 800
Training R²: 0.7818680441699353
Testing R²: 0.7625330384687853
Training Mape: 0.13648878302949877
Testing Mape: 0.14412276511303324
chunk_size: 1000
Training R²: 0.7385559595092921
Testing R²: 0.7202272439900359
Training Mape: 0.15788319995736125
Testing Mape: 0.16059400866991194


In [12]:
# x86_ecore 1.5GHz -> 3.0GHz
import pandas as pd
import math
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, VotingRegressor
from sklearn.metrics import r2_score
import pandas as pd
import math
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, VotingRegressor
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_percentage_error

def process_csv_feautures(file_paths, chunk_size):
    all_features = []
    for file_path in file_paths:
        df = pd.read_csv(file_path + '_' + str(chunk_size))
        df = df.apply(pd.to_numeric, errors='coerce').fillna(0)
        features = []
        for index, row in df.iterrows():
            inst = row['cpu_core/instructions/u']
            cyc = row['cpu_core/cpu-cycles/u']
            ipc = inst/cyc
            features.append([
            row['cpu_core/instructions/u'],
            row['cpu_core/cpu-cycles/u'],
            row['cpu_core/br_inst_retired.all_branches/u'],
            row['cpu_core/br_misp_retired.all_branches/u'],
            row['cpu_core/L1-dcache-load-misses:u/'],
            row['cpu_core/L1-dcache-loads:u/'],
            row['cpu_core/L1-dcache-stores:u/'],
            row['cpu_core/L1-icache-load-misses:u/'],
            row['cpu_core/LLC-load-misses:u/'],
            row['cpu_core/LLC-loads:u/'],
            row['cpu_core/LLC-store-misses:u/'],
            row['cpu_core/LLC-stores:u/'],
            row['cpu_core/fp_arith_inst_retired.scalar_single/u'],
            row['cpu_core/dTLB-load-misses:u/'],
            row['cpu_core/dTLB-loads:u/'],
            row['cpu_core/dTLB-store-misses:u/'],
            row['cpu_core/dTLB-stores:u/'],
            row['cpu_core/iTLB-load-misses:u/'],
            row['cpu_core/bus-cycles/u'],
            row['cpu_core/mem_load_retired.l1_hit/u'],
            row['cpu_core/mem_load_retired.l1_miss/u'],
            row['cpu_core/mem_load_retired.l2_hit/u'],
            row['cpu_core/mem_load_retired.l2_miss/u'],
            row['cpu_core/mem_load_retired.l3_hit/u'],
            row['cpu_core/mem_load_retired.l3_miss/u'],
            ipc])
        all_features.extend(features)
    return all_features


def process_csv_ipc(file_paths, chunk_size):
    all_ipc_values = []
    for file_path in file_paths:
        df = pd.read_csv(file_path + '_' + str(chunk_size))
        df = df.apply(pd.to_numeric, errors='coerce').fillna(0)
        for _, row in df.iterrows():
            all_ipc_values.append(row['cpu_core/instructions/u'] / row['cpu_core/cpu-cycles/u'])  
    return all_ipc_values

one_ghz_files = [
    '../../Data/traces/inst_aligned_traces/x86_desktop_heterogeneous/x86_desktop_heterogeneous_pcore_1.5GHz_500',
    '../../Data/traces/inst_aligned_traces/x86_desktop_heterogeneous/x86_desktop_heterogeneous_pcore_1.5GHz_502',
    '../../Data/traces/inst_aligned_traces/x86_desktop_heterogeneous/x86_desktop_heterogeneous_pcore_1.5GHz_505',
    '../../Data/traces/inst_aligned_traces/x86_desktop_heterogeneous/x86_desktop_heterogeneous_pcore_1.5GHz_520',
    '../../Data/traces/inst_aligned_traces/x86_desktop_heterogeneous/x86_desktop_heterogeneous_pcore_1.5GHz_523',
    '../../Data/traces/inst_aligned_traces/x86_desktop_heterogeneous/x86_desktop_heterogeneous_pcore_1.5GHz_525',
    '../../Data/traces/inst_aligned_traces/x86_desktop_heterogeneous/x86_desktop_heterogeneous_pcore_1.5GHz_531',
    '../../Data/traces/inst_aligned_traces/x86_desktop_heterogeneous/x86_desktop_heterogeneous_pcore_1.5GHz_541',
    '../../Data/traces/inst_aligned_traces/x86_desktop_heterogeneous/x86_desktop_heterogeneous_pcore_1.5GHz_548',
    '../../Data/traces/inst_aligned_traces/x86_desktop_heterogeneous/x86_desktop_heterogeneous_pcore_1.5GHz_557',
    '../../Data/traces/inst_aligned_traces/x86_desktop_heterogeneous/x86_desktop_heterogeneous_pcore_1.5GHz_503',
    '../../Data/traces/inst_aligned_traces/x86_desktop_heterogeneous/x86_desktop_heterogeneous_pcore_1.5GHz_507',
    '../../Data/traces/inst_aligned_traces/x86_desktop_heterogeneous/x86_desktop_heterogeneous_pcore_1.5GHz_508',
    '../../Data/traces/inst_aligned_traces/x86_desktop_heterogeneous/x86_desktop_heterogeneous_pcore_1.5GHz_510',
    '../../Data/traces/inst_aligned_traces/x86_desktop_heterogeneous/x86_desktop_heterogeneous_pcore_1.5GHz_511',
    '../../Data/traces/inst_aligned_traces/x86_desktop_heterogeneous/x86_desktop_heterogeneous_pcore_1.5GHz_519',
    '../../Data/traces/inst_aligned_traces/x86_desktop_heterogeneous/x86_desktop_heterogeneous_pcore_1.5GHz_521',
    '../../Data/traces/inst_aligned_traces/x86_desktop_heterogeneous/x86_desktop_heterogeneous_pcore_1.5GHz_527',
    '../../Data/traces/inst_aligned_traces/x86_desktop_heterogeneous/x86_desktop_heterogeneous_pcore_1.5GHz_538',
    '../../Data/traces/inst_aligned_traces/x86_desktop_heterogeneous/x86_desktop_heterogeneous_pcore_1.5GHz_544',
    '../../Data/traces/inst_aligned_traces/x86_desktop_heterogeneous/x86_desktop_heterogeneous_pcore_1.5GHz_549',
    '../../Data/traces/inst_aligned_traces/x86_desktop_heterogeneous/x86_desktop_heterogeneous_pcore_1.5GHz_554',
]
    
three_ghz_files = [
    '../../Data/traces/inst_aligned_traces/x86_desktop_heterogeneous/x86_desktop_heterogeneous_pcore_3.0GHz_500',
    '../../Data/traces/inst_aligned_traces/x86_desktop_heterogeneous/x86_desktop_heterogeneous_pcore_3.0GHz_502',
    '../../Data/traces/inst_aligned_traces/x86_desktop_heterogeneous/x86_desktop_heterogeneous_pcore_3.0GHz_505',
    '../../Data/traces/inst_aligned_traces/x86_desktop_heterogeneous/x86_desktop_heterogeneous_pcore_3.0GHz_520',
    '../../Data/traces/inst_aligned_traces/x86_desktop_heterogeneous/x86_desktop_heterogeneous_pcore_3.0GHz_523',
    '../../Data/traces/inst_aligned_traces/x86_desktop_heterogeneous/x86_desktop_heterogeneous_pcore_3.0GHz_525',
    '../../Data/traces/inst_aligned_traces/x86_desktop_heterogeneous/x86_desktop_heterogeneous_pcore_3.0GHz_531',
    '../../Data/traces/inst_aligned_traces/x86_desktop_heterogeneous/x86_desktop_heterogeneous_pcore_3.0GHz_541',
    '../../Data/traces/inst_aligned_traces/x86_desktop_heterogeneous/x86_desktop_heterogeneous_pcore_3.0GHz_548',
    '../../Data/traces/inst_aligned_traces/x86_desktop_heterogeneous/x86_desktop_heterogeneous_pcore_3.0GHz_557',
    '../../Data/traces/inst_aligned_traces/x86_desktop_heterogeneous/x86_desktop_heterogeneous_pcore_3.0GHz_503',
    '../../Data/traces/inst_aligned_traces/x86_desktop_heterogeneous/x86_desktop_heterogeneous_pcore_3.0GHz_507',
    '../../Data/traces/inst_aligned_traces/x86_desktop_heterogeneous/x86_desktop_heterogeneous_pcore_3.0GHz_508',
    '../../Data/traces/inst_aligned_traces/x86_desktop_heterogeneous/x86_desktop_heterogeneous_pcore_3.0GHz_510',
    '../../Data/traces/inst_aligned_traces/x86_desktop_heterogeneous/x86_desktop_heterogeneous_pcore_3.0GHz_511',
    '../../Data/traces/inst_aligned_traces/x86_desktop_heterogeneous/x86_desktop_heterogeneous_pcore_3.0GHz_519',
    '../../Data/traces/inst_aligned_traces/x86_desktop_heterogeneous/x86_desktop_heterogeneous_pcore_3.0GHz_521',
    '../../Data/traces/inst_aligned_traces/x86_desktop_heterogeneous/x86_desktop_heterogeneous_pcore_3.0GHz_527',
    '../../Data/traces/inst_aligned_traces/x86_desktop_heterogeneous/x86_desktop_heterogeneous_pcore_3.0GHz_538',
    '../../Data/traces/inst_aligned_traces/x86_desktop_heterogeneous/x86_desktop_heterogeneous_pcore_3.0GHz_544',
    '../../Data/traces/inst_aligned_traces/x86_desktop_heterogeneous/x86_desktop_heterogeneous_pcore_3.0GHz_549',
    '../../Data/traces/inst_aligned_traces/x86_desktop_heterogeneous/x86_desktop_heterogeneous_pcore_3.0GHz_554',
]

chunks = [200, 400, 600, 800, 1000]
for chunk_size in chunks:
    in_order_hardware_counters_features = process_csv_feautures(one_ghz_files, chunk_size)
    out_of_order_ipc = process_csv_ipc(three_ghz_files, chunk_size)
    print('chunk_size: ' + str(chunk_size))
    
    data = {
        'In Order': in_order_hardware_counters_features,
        'Out of Order': out_of_order_ipc
    }
    
    length = min(len(in_order_hardware_counters_features), len(out_of_order_ipc))
    
    # Shorten the longen array
    in_order_hardware_counters_features = in_order_hardware_counters_features[:length]
    out_of_order_ipc = out_of_order_ipc[:length]
    
    # separate features and target
    X = pd.DataFrame(
        in_order_hardware_counters_features, 
        columns=[
            'cpu_core/instructions/u',
            'cpu_core/cpu-cycles/u',
            'cpu_core/br_inst_retired.all_branches/u',
            'cpu_core/br_misp_retired.all_branches/u',
            'cpu_core/L1-dcache-load-misses:u/',
            'cpu_core/L1-dcache-loads:u/',
            'cpu_core/L1-dcache-stores:u/',
            'cpu_core/L1-icache-load-misses:u/',
            'cpu_core/LLC-load-misses:u/',
            'cpu_core/LLC-loads:u/',
            'cpu_core/LLC-store-misses:u/',
            'cpu_core/LLC-stores:u/',
            'cpu_core/fp_arith_inst_retired.scalar_single/u',
            'cpu_core/dTLB-load-misses:u/',
            'cpu_core/dTLB-loads:u/',
            'cpu_core/dTLB-store-misses:u/',
            'cpu_core/dTLB-stores:u/',
            'cpu_core/iTLB-load-misses:u/',
            'cpu_core/bus-cycles/u',
            'cpu_core/mem_load_retired.l1_hit/u',
            'cpu_core/mem_load_retired.l1_miss/u',
            'cpu_core/mem_load_retired.l2_hit/u',
            'cpu_core/mem_load_retired.l2_miss/u',
            'cpu_core/mem_load_retired.l3_hit/u',
            'cpu_core/mem_load_retired.l3_miss/u',
            'ipc'])  # Features
    y = pd.Series(out_of_order_ipc) 
    
    # split 
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
    
    rf_model = RandomForestRegressor(n_estimators=100)
    gb_model = GradientBoostingRegressor(n_estimators=100)
    ensemble_model = gb_model
    #ensemble_model = VotingRegressor([('rf', rf_model), ('gb', gb_model)])
    ensemble_model.fit(X_train, y_train)
    
    ensemble_predictions = ensemble_model.predict(X_test)
    # Calculate R²
    ensemble_r2 = r2_score(y_test, ensemble_predictions)
    
    #bruh wut
    train_predictions = ensemble_model.predict(X_train)
    test_predictions = ensemble_model.predict(X_test)
    
    # Calculate R² for training and testing sets
    train_r2 = r2_score(y_train, train_predictions)
    train_mape = mean_absolute_percentage_error(y_train, train_predictions)
    test_r2 = r2_score(y_test, test_predictions)
    test_mape = mean_absolute_percentage_error(y_test, test_predictions)
    
    
    print(f"Training R²: {train_r2}")
    print(f"Testing R²: {test_r2}")
    print(f"Training Mape: {train_mape}")
    print(f"Testing Mape: {test_mape}")


chunk_size: 200
Training R²: 0.9560470563545663
Testing R²: 0.9255525402664134
Training Mape: 0.0450529952593073
Testing Mape: 0.05637267324389405
chunk_size: 400
Training R²: 0.8896304181057711
Testing R²: 0.8465173194483822
Training Mape: 0.07800811545408294
Testing Mape: 0.0868265128703729
chunk_size: 600
Training R²: 0.8492545980282633
Testing R²: 0.8189892622637436
Training Mape: 0.09412536875675513
Testing Mape: 0.10120378556583262
chunk_size: 800
Training R²: 0.8070003098042939
Testing R²: 0.7725583947632606
Training Mape: 0.11068757209888279
Testing Mape: 0.11259754179834551
chunk_size: 1000
Training R²: 0.773445648453028
Testing R²: 0.7597767890391395
Training Mape: 0.12265325407313415
Testing Mape: 0.12783495210370652


In [13]:
# x86_ecore 3.0GHz -> 1.5GHz
import pandas as pd
import math
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, VotingRegressor
from sklearn.metrics import r2_score
import pandas as pd
import math
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, VotingRegressor
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_percentage_error

def process_csv_feautures(file_paths, chunk_size):
    all_features = []
    for file_path in file_paths:
        df = pd.read_csv(file_path + '_' + str(chunk_size))
        df = df.apply(pd.to_numeric, errors='coerce').fillna(0)
        features = []
        for index, row in df.iterrows():
            inst = row['cpu_core/instructions/u']
            cyc = row['cpu_core/cpu-cycles/u']
            ipc = inst/cyc
            features.append([
            row['cpu_core/instructions/u'],
            row['cpu_core/cpu-cycles/u'],
            row['cpu_core/br_inst_retired.all_branches/u'],
            row['cpu_core/br_misp_retired.all_branches/u'],
            row['cpu_core/L1-dcache-load-misses:u/'],
            row['cpu_core/L1-dcache-loads:u/'],
            row['cpu_core/L1-dcache-stores:u/'],
            row['cpu_core/L1-icache-load-misses:u/'],
            row['cpu_core/LLC-load-misses:u/'],
            row['cpu_core/LLC-loads:u/'],
            row['cpu_core/LLC-store-misses:u/'],
            row['cpu_core/LLC-stores:u/'],
            row['cpu_core/fp_arith_inst_retired.scalar_single/u'],
            row['cpu_core/dTLB-load-misses:u/'],
            row['cpu_core/dTLB-loads:u/'],
            row['cpu_core/dTLB-store-misses:u/'],
            row['cpu_core/dTLB-stores:u/'],
            row['cpu_core/iTLB-load-misses:u/'],
            row['cpu_core/bus-cycles/u'],
            row['cpu_core/mem_load_retired.l1_hit/u'],
            row['cpu_core/mem_load_retired.l1_miss/u'],
            row['cpu_core/mem_load_retired.l2_hit/u'],
            row['cpu_core/mem_load_retired.l2_miss/u'],
            row['cpu_core/mem_load_retired.l3_hit/u'],
            row['cpu_core/mem_load_retired.l3_miss/u'],
            ipc])
        all_features.extend(features)
    return all_features


def process_csv_ipc(file_paths, chunk_size):
    all_ipc_values = []
    for file_path in file_paths:
        df = pd.read_csv(file_path + '_' + str(chunk_size))
        df = df.apply(pd.to_numeric, errors='coerce').fillna(0)
        for _, row in df.iterrows():
            all_ipc_values.append(row['cpu_core/instructions/u'] / row['cpu_core/cpu-cycles/u'])  
    return all_ipc_values

one_ghz_files = [
    '../../Data/traces/inst_aligned_traces/x86_desktop_heterogeneous/x86_desktop_heterogeneous_pcore_1.5GHz_500',
    '../../Data/traces/inst_aligned_traces/x86_desktop_heterogeneous/x86_desktop_heterogeneous_pcore_1.5GHz_502',
    '../../Data/traces/inst_aligned_traces/x86_desktop_heterogeneous/x86_desktop_heterogeneous_pcore_1.5GHz_505',
    '../../Data/traces/inst_aligned_traces/x86_desktop_heterogeneous/x86_desktop_heterogeneous_pcore_1.5GHz_520',
    '../../Data/traces/inst_aligned_traces/x86_desktop_heterogeneous/x86_desktop_heterogeneous_pcore_1.5GHz_523',
    '../../Data/traces/inst_aligned_traces/x86_desktop_heterogeneous/x86_desktop_heterogeneous_pcore_1.5GHz_525',
    '../../Data/traces/inst_aligned_traces/x86_desktop_heterogeneous/x86_desktop_heterogeneous_pcore_1.5GHz_531',
    '../../Data/traces/inst_aligned_traces/x86_desktop_heterogeneous/x86_desktop_heterogeneous_pcore_1.5GHz_541',
    '../../Data/traces/inst_aligned_traces/x86_desktop_heterogeneous/x86_desktop_heterogeneous_pcore_1.5GHz_548',
    '../../Data/traces/inst_aligned_traces/x86_desktop_heterogeneous/x86_desktop_heterogeneous_pcore_1.5GHz_557',
    '../../Data/traces/inst_aligned_traces/x86_desktop_heterogeneous/x86_desktop_heterogeneous_pcore_1.5GHz_503',
    '../../Data/traces/inst_aligned_traces/x86_desktop_heterogeneous/x86_desktop_heterogeneous_pcore_1.5GHz_507',
    '../../Data/traces/inst_aligned_traces/x86_desktop_heterogeneous/x86_desktop_heterogeneous_pcore_1.5GHz_508',
    '../../Data/traces/inst_aligned_traces/x86_desktop_heterogeneous/x86_desktop_heterogeneous_pcore_1.5GHz_510',
    '../../Data/traces/inst_aligned_traces/x86_desktop_heterogeneous/x86_desktop_heterogeneous_pcore_1.5GHz_511',
    '../../Data/traces/inst_aligned_traces/x86_desktop_heterogeneous/x86_desktop_heterogeneous_pcore_1.5GHz_519',
    '../../Data/traces/inst_aligned_traces/x86_desktop_heterogeneous/x86_desktop_heterogeneous_pcore_1.5GHz_521',
    '../../Data/traces/inst_aligned_traces/x86_desktop_heterogeneous/x86_desktop_heterogeneous_pcore_1.5GHz_527',
    '../../Data/traces/inst_aligned_traces/x86_desktop_heterogeneous/x86_desktop_heterogeneous_pcore_1.5GHz_538',
    '../../Data/traces/inst_aligned_traces/x86_desktop_heterogeneous/x86_desktop_heterogeneous_pcore_1.5GHz_544',
    '../../Data/traces/inst_aligned_traces/x86_desktop_heterogeneous/x86_desktop_heterogeneous_pcore_1.5GHz_549',
    '../../Data/traces/inst_aligned_traces/x86_desktop_heterogeneous/x86_desktop_heterogeneous_pcore_1.5GHz_554',
]
    
three_ghz_files = [
    '../../Data/traces/inst_aligned_traces/x86_desktop_heterogeneous/x86_desktop_heterogeneous_pcore_3.0GHz_500',
    '../../Data/traces/inst_aligned_traces/x86_desktop_heterogeneous/x86_desktop_heterogeneous_pcore_3.0GHz_502',
    '../../Data/traces/inst_aligned_traces/x86_desktop_heterogeneous/x86_desktop_heterogeneous_pcore_3.0GHz_505',
    '../../Data/traces/inst_aligned_traces/x86_desktop_heterogeneous/x86_desktop_heterogeneous_pcore_3.0GHz_520',
    '../../Data/traces/inst_aligned_traces/x86_desktop_heterogeneous/x86_desktop_heterogeneous_pcore_3.0GHz_523',
    '../../Data/traces/inst_aligned_traces/x86_desktop_heterogeneous/x86_desktop_heterogeneous_pcore_3.0GHz_525',
    '../../Data/traces/inst_aligned_traces/x86_desktop_heterogeneous/x86_desktop_heterogeneous_pcore_3.0GHz_531',
    '../../Data/traces/inst_aligned_traces/x86_desktop_heterogeneous/x86_desktop_heterogeneous_pcore_3.0GHz_541',
    '../../Data/traces/inst_aligned_traces/x86_desktop_heterogeneous/x86_desktop_heterogeneous_pcore_3.0GHz_548',
    '../../Data/traces/inst_aligned_traces/x86_desktop_heterogeneous/x86_desktop_heterogeneous_pcore_3.0GHz_557',
    '../../Data/traces/inst_aligned_traces/x86_desktop_heterogeneous/x86_desktop_heterogeneous_pcore_3.0GHz_503',
    '../../Data/traces/inst_aligned_traces/x86_desktop_heterogeneous/x86_desktop_heterogeneous_pcore_3.0GHz_507',
    '../../Data/traces/inst_aligned_traces/x86_desktop_heterogeneous/x86_desktop_heterogeneous_pcore_3.0GHz_508',
    '../../Data/traces/inst_aligned_traces/x86_desktop_heterogeneous/x86_desktop_heterogeneous_pcore_3.0GHz_510',
    '../../Data/traces/inst_aligned_traces/x86_desktop_heterogeneous/x86_desktop_heterogeneous_pcore_3.0GHz_511',
    '../../Data/traces/inst_aligned_traces/x86_desktop_heterogeneous/x86_desktop_heterogeneous_pcore_3.0GHz_519',
    '../../Data/traces/inst_aligned_traces/x86_desktop_heterogeneous/x86_desktop_heterogeneous_pcore_3.0GHz_521',
    '../../Data/traces/inst_aligned_traces/x86_desktop_heterogeneous/x86_desktop_heterogeneous_pcore_3.0GHz_527',
    '../../Data/traces/inst_aligned_traces/x86_desktop_heterogeneous/x86_desktop_heterogeneous_pcore_3.0GHz_538',
    '../../Data/traces/inst_aligned_traces/x86_desktop_heterogeneous/x86_desktop_heterogeneous_pcore_3.0GHz_544',
    '../../Data/traces/inst_aligned_traces/x86_desktop_heterogeneous/x86_desktop_heterogeneous_pcore_3.0GHz_549',
    '../../Data/traces/inst_aligned_traces/x86_desktop_heterogeneous/x86_desktop_heterogeneous_pcore_3.0GHz_554',
]

chunks = [200, 400, 600, 800, 1000]
for chunk_size in chunks:
    in_order_hardware_counters_features = process_csv_feautures(three_ghz_files, chunk_size)
    out_of_order_ipc = process_csv_ipc(one_ghz_files, chunk_size)
    print('chunk_size: ' + str(chunk_size))
    
    data = {
        'In Order': in_order_hardware_counters_features,
        'Out of Order': out_of_order_ipc
    }
    
    length = min(len(in_order_hardware_counters_features), len(out_of_order_ipc))
    
    # Shorten the longen array
    in_order_hardware_counters_features = in_order_hardware_counters_features[:length]
    out_of_order_ipc = out_of_order_ipc[:length]
    
    # separate features and target
    X = pd.DataFrame(
        in_order_hardware_counters_features, 
        columns=[
            'cpu_core/instructions/u',
            'cpu_core/cpu-cycles/u',
            'cpu_core/br_inst_retired.all_branches/u',
            'cpu_core/br_misp_retired.all_branches/u',
            'cpu_core/L1-dcache-load-misses:u/',
            'cpu_core/L1-dcache-loads:u/',
            'cpu_core/L1-dcache-stores:u/',
            'cpu_core/L1-icache-load-misses:u/',
            'cpu_core/LLC-load-misses:u/',
            'cpu_core/LLC-loads:u/',
            'cpu_core/LLC-store-misses:u/',
            'cpu_core/LLC-stores:u/',
            'cpu_core/fp_arith_inst_retired.scalar_single/u',
            'cpu_core/dTLB-load-misses:u/',
            'cpu_core/dTLB-loads:u/',
            'cpu_core/dTLB-store-misses:u/',
            'cpu_core/dTLB-stores:u/',
            'cpu_core/iTLB-load-misses:u/',
            'cpu_core/bus-cycles/u',
            'cpu_core/mem_load_retired.l1_hit/u',
            'cpu_core/mem_load_retired.l1_miss/u',
            'cpu_core/mem_load_retired.l2_hit/u',
            'cpu_core/mem_load_retired.l2_miss/u',
            'cpu_core/mem_load_retired.l3_hit/u',
            'cpu_core/mem_load_retired.l3_miss/u',
            'ipc'])  # Features
    y = pd.Series(out_of_order_ipc) 
    
    # split 
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
    
    rf_model = RandomForestRegressor(n_estimators=100)
    gb_model = GradientBoostingRegressor(n_estimators=100)
    ensemble_model = gb_model
    #ensemble_model = VotingRegressor([('rf', rf_model), ('gb', gb_model)])
    ensemble_model.fit(X_train, y_train)
    
    ensemble_predictions = ensemble_model.predict(X_test)
    # Calculate R²
    ensemble_r2 = r2_score(y_test, ensemble_predictions)
    
    #bruh wut
    train_predictions = ensemble_model.predict(X_train)
    test_predictions = ensemble_model.predict(X_test)
    
    # Calculate R² for training and testing sets
    train_r2 = r2_score(y_train, train_predictions)
    train_mape = mean_absolute_percentage_error(y_train, train_predictions)
    test_r2 = r2_score(y_test, test_predictions)
    test_mape = mean_absolute_percentage_error(y_test, test_predictions)
    
    
    print(f"Training R²: {train_r2}")
    print(f"Testing R²: {test_r2}")
    print(f"Training Mape: {train_mape}")
    print(f"Testing Mape: {test_mape}")


chunk_size: 200
Training R²: 0.9591827446750651
Testing R²: 0.9204749306189509
Training Mape: 0.04449874217067498
Testing Mape: 0.05359190074785353
chunk_size: 400
Training R²: 0.8806294048529189
Testing R²: 0.8220766336547698
Training Mape: 0.07510497456777923
Testing Mape: 0.08767650541695927
chunk_size: 600
Training R²: 0.8294097883049338
Testing R²: 0.7936978537453534
Training Mape: 0.09651089508670929
Testing Mape: 0.09932749698726898
chunk_size: 800
Training R²: 0.7941300964274606
Testing R²: 0.7592647736532548
Training Mape: 0.10982938480415425
Testing Mape: 0.11439025080661011
chunk_size: 1000
Training R²: 0.7548186454964605
Testing R²: 0.7472125083061545
Training Mape: 0.12180887832931156
Testing Mape: 0.12427665074873566
